In [10]:
import os,sys
sys.path.append('/tf/crypto_prediction_ml_dl/script')
from trino_operations import trino_operator
import pandas as pd
import numpy as np
from datetime import datetime
from pprint import pprint
import copy
!echo "pwd: `pwd`"

pwd: /tf/crypto_prediction_ml_dl/notebook/TFT_models


In [11]:
# Create a new folder for new crypto dataset
path = "./day_models/data/crypto_day_all"
isExist = os.path.exists(path)
if not isExist:
   os.makedirs(path)
   print("The new directory is created!")
    
# The dataset is gonna be saved as out_file.
out_file = os.path.join(path, 'crypto_day_all.csv')

In [12]:
# Load crypto data from Hive mart tables via Trino
trino_ope = trino_operator.Operator()

def create_dataframe_from_hive_mart_table(
    target_schema, target_table, target_columns, target_symbol, column_prefix
):
    query = f"""
    select
        row_number() over(order by dt) as row_id,
        {target_columns}
    from 
        hive.{target_schema}.{target_table}
    where
        id = '{target_symbol}'
    order by dt
    """

    res = trino_ope.run_query(query)

    indicators_query_result = {}
    for idx, row_data in enumerate(res,1):
        indicators_query_result[int(idx)] = row_data

    _target_columns_list = ['row_id'] + target_columns.strip().split(",")
    target_columns_list = [
        column_prefix + column.strip() for column in _target_columns_list
    ]
    indicators_raw_df = pd.DataFrame.from_dict(
        indicators_query_result, orient="index", columns=target_columns_list
    )

    return indicators_raw_df

##################################
# Extract dataset from hive tables
##################################

# Crypto
target_schema = "crypto_mart"
target_table = "crypto_indicator_day"
target_symbol = "BTC_USDT"
target_columns = """
    id,
    dt,
    low,
    high,
    open,
    close,
    volume,
    macd,
    macd_single,
    rsi,
    bollinger_bands_sma,
    bollinger_bands_lower_band,
    bollinger_bands_upper_band,
    obv,
    obv_sma,
    ichimoku_chikou_span,
    ichimoku_kijun_sen,
    ichimoku_tenkan_sen,
    ichimoku_senkou_span_a,
    ichimoku_senkou_span_b,
    stoch_oscillator,
    stoch_signal,
    stoch_percent_j,
    aroon_up,
    aroon_down,
    aroon_oscillator,
    sma5,
    sma10,
    sma30,
    ema5,
    ema10,
    ema30
    """

column_prefix = "btc_"
btc_raw_df = create_dataframe_from_hive_mart_table(
    target_schema, target_table, target_columns, target_symbol, column_prefix
)

##################
# Create Dataset #
##################
# Create target value for prediction
dt_column = btc_raw_df["btc_dt"]

# Check if all "date" exist in the dataframe.
first_dt = str(btc_raw_df.iloc[0]['btc_dt'])
last_dt = str(btc_raw_df.iloc[-1]['btc_dt'])
desired_date_range = pd.date_range(start=first_dt, end=last_dt, freq='D')
all_dates_exist = all(str(date)[:11].strip() == str(btc_raw_df.iloc[i]['btc_dt']).strip() for i,date in enumerate(desired_date_range))
if all_dates_exist:
    print("All dates exist in the DataFrame's index.")
else:
    print("Not all dates exist in the DataFrame's index.")

# btc_dataset_df = btc_raw_df[["btc_low","btc_high","btc_open","btc_close","btc_volume"]]
btc_raw_df.drop(["btc_dt"], axis=1)

predicting_days = 1 # predicting after N day price

##################
# Create Dataset #
##################
# Create target value for prediction
btc_df_with_target = btc_raw_df
target = btc_raw_df['btc_close'].shift(-predicting_days)
btc_df_with_target['target'] = target

# Replace NaN value to forward or backward values.
btc_df_with_target = btc_df_with_target.fillna(method='ffill')
btc_df_with_target = btc_df_with_target.fillna(method='bfill')

# Get the columns with NaN values
nan_columns = btc_df_with_target.columns[btc_df_with_target.isna().any()].tolist()

# Drop columns containing NaN values even after filling the value.
btc_df_with_target = btc_df_with_target.dropna(axis=1)

# NULL(NaN) check: False means no NULL data, True means contains NULL data
if btc_df_with_target.isnull().any().any():
    print("ERROR: NULL is in the dataset")

btc_df_with_target.to_csv(out_file)
btc_df_with_target

All dates exist in the DataFrame's index.


,btc_row_id,btc_id,btc_dt,btc_low,btc_high,btc_open,btc_close,btc_volume,btc_macd,btc_macd_single,...,btc_aroon_up,btc_aroon_down,btc_aroon_oscillator,btc_sma5,btc_sma10,btc_sma30,btc_ema5,btc_ema10,btc_ema30,target
1,1,BTC_USDT,2020-11-01,13620.57,13888.45,13790.61,13747.25,9018406.0,1341.0406,1139.0571,...,96.0,4.0,92.0,14208.216,14752.828,16656.650,14208.216,14752.828,16656.650,13558.84
2,2,BTC_USDT,2020-11-02,13207.51,13825.00,13749.99,13558.84,19482722.0,1341.0406,1139.0571,...,96.0,4.0,92.0,14208.216,14752.828,16656.650,14208.216,14752.828,16656.650,14014.59
3,3,BTC_USDT,2020-11-03,13287.61,14046.57,13556.05,14014.59,17432730.0,1341.0406,1139.0571,...,96.0,4.0,92.0,14208.216,14752.828,16656.650,14208.216,14752.828,16656.650,14133.45
4,4,BTC_USDT,2020-11-04,13532.19,14240.00,14014.59,14133.45,19503998.0,1341.0406,1139.0571,...,96.0,4.0,92.0,14208.216,14752.828,16656.650,14208.216,14752.828,16656.650,15586.95
5,5,BTC_USDT,2020-11-05,14092.32,15740.00,14143.70,15586.95,38253920.0,1341.0406,1139.0571,...,96.0,4.0,92.0,14208.216,14752.828,16656.650,14208.216,14752.828,16656.650,15579.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,1111,BTC_USDT,2023-11-16,35627.85,37875.00,37825.30,36173.01,144074128.0,1460.2429,1623.2957,...,72.0,4.0,72.0,36609.254,36445.530,34277.180,36563.170,36286.484,34247.047,36610.56
1112,1112,BTC_USDT,2023-11-17,35897.26,36681.10,36166.19,36610.56,152239328.0,1391.1339,1576.8634,...,68.0,4.0,68.0,36520.510,36567.418,34553.605,36578.965,36345.406,34399.530,36557.17
1113,1113,BTC_USDT,2023-11-18,36205.97,36828.05,36610.69,36557.17,141763952.0,1316.8765,1524.8660,...,64.0,4.0,64.0,36548.348,36661.188,34815.023,36571.703,36383.910,34538.734,37315.46
1114,1114,BTC_USDT,2023-11-19,36400.00,37430.03,36557.18,37315.46,101434232.0,1304.1808,1480.7290,...,60.0,8.0,52.0,36898.200,36723.254,35070.066,36819.620,36553.285,34717.880,37290.27
